In [6]:
from PIL import Image
import tensorflow as tf
import mnist
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix

Making Variables after importing dataset.

In [ ]:
#Let's create the training and testing variables
 #   X_train = mnist.train_images()
 #   y_train = mnist.train_labels()

 #   X_test = mnist.test_images()
 #   y_test = mnist.test_labels()

#This wont work as the mnist library is trying to download the MNIST dataset from a URL that is no longer valid or accessible. So use Tenserflow/Keras instead to load MNIST. 

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

#Normalising data to range [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 8s 1us/step


In [21]:
#Let's look at the data stored in the Var:
print('X_train', X_train)
print('X_test', X_test)
print('y_train', y_train) 
print('y_test', y_test) 

#X_train, X_test are 3 dimensional array while y_train and y_test are just a list of digits from 0 to 9. To check dimension of the data:
print("\nThe dimension of X is:", X_train.ndim) 

X_train [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
X_

In [22]:
print(X_train.shape)

(60000, 28, 28)


So we have 60000 instances of data consisting of 28 x 28 Matrices(Pixels).